# Import everything

In [1]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO
import numpy as np
from scipy import signal
from scipy import fftpack
import seaborn as sns
from tqdm.notebook import tqdm
import math

import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import multiprocessing
from joblib import Parallel, delayed
import multiprocessing
from joblib import wrap_non_picklable_objects
import json
import pickle
import os.path
from mpl_toolkits.mplot3d import axes3d
import timeit
from timeit import default_timer as timer
from datetime import timedelta
import json
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod import bayes_mixed_glm as glm
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFECV
from spectrum import arburg, arma2psd, pburg
import pylab
from scipy.signal import find_peaks, butter
from scipy.integrate import simps
from scipy.io import loadmat
from numpy import trapz
import scipy.fftpack

# Load file paths of EEGs.

In [2]:
paths_raw = sorted(glob.glob('data/dataset3/raw/*.mat'))
paths_raw_hjorth = sorted(glob.glob('data/dataset3/raw-hjorth/*.mat'))
paths_clean = sorted(glob.glob('data/dataset3/clean/*.mat'))
paths_clean_hjorth = sorted(glob.glob('data/dataset3/clean-hjorth/*.mat'))
print("Raw: {}\nRaw Hjorth: {}\nClean: {}\nClean Hjorth: {}".format(len(paths_raw), len(paths_raw_hjorth), len(paths_clean), len(paths_clean_hjorth)))

Raw: 15
Raw Hjorth: 15
Clean: 15
Clean Hjorth: 15


# Read EEG files

In [3]:
def read_trials_from_raw(filename):
    x = loadmat(filename)
    x = x['EEGData']
    trials = []
    time = np.linspace(-1000, 0, len(x[0, :, 0]))
    for trial_num in range(x.shape[2]):
        trial = np.transpose(x[:, :, trial_num])
        trial = pd.DataFrame(data=trial, columns=['FP1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'IZ', 'OZ', 'POZ', 'PZ', 'CPZ', 'FPZ', 'FP2', 'AF8', 'AF4', 'AFZ', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCZ', 'CZ', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'M1', 'M2', 'NAS', 'LVEOG', 'RVEOG', 'LHEOG', 'RHEOG', 'NFPZ'])
        trial['time'] = time
        trials.append(trial)
    return trials

def read_trials_from_hjorth(filename):
    x = loadmat(filename)
    mat_trials = x['dat'][0][0][3][0]
    trials = []
    time = np.linspace(-1000, 0, len(mat_trials[0][0]))
    for mat_trial in mat_trials:
        trials.append(pd.DataFrame({'C3': mat_trial[0], 'C4': mat_trial[1], 'time': time}))
    return trials

# Power and phase calculation

In [4]:
def blackman_harris_filter(channel, time, cutoffs, fs, numtaps=801):
    b = signal.firwin(numtaps, cutoffs, window='blackmanharris', fs=fs)
    filtered = signal.lfilter(b, 1, channel)
    delay = 0.5 * (numtaps - 1) / fs
    df = pd.DataFrame({
        'time': time-delay,
        'channel': filtered
    })
    return df

def butter_bandpass_filter(data, lowcut, highcut, fs, btype='bandpass', order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype=btype)
    y = signal.lfilter(b, a, data)
    return y

def get_phase(channel, time, band, fs, filter_type, start_time_ms, stop_time_ms, blackmann_harris_ntaps):
    if filter_type=='butter':
        df_filtered = pd.DataFrame({'channel': butter_bandpass_filter(channel, band[0], band[1], fs, btype='bandpass', order=3), 'time': time})
    else:
        df_filtered = blackman_harris_filter(channel, time, [0.000001, band[0]], fs, numtaps=blackmann_harris_ntaps)
        df_filtered = blackman_harris_filter(df_filtered['channel'].values, df_filtered['time'].values, band[1], fs, numtaps=blackmann_harris_ntaps)    
    
    hilb = signal.hilbert(df_filtered[(df_filtered['time'] > start_time_ms) * (df_filtered['time'] < stop_time_ms)]['channel'])
    phase = np.angle(hilb, deg=True)
    df_phase = pd.DataFrame(phase, columns=['phase'])
    df_phase['time'] = df_filtered[(df_filtered['time'] > start_time_ms) * (df_filtered['time'] < stop_time_ms)]['time'].values
    return df_phase

def get_power(channel, time, fs, crop_start_millis, crop_end_millis, method, filter_type, resample_fs, line_noise_band, blackmann_harris_ntaps):       
    # Crop.
    channel = channel[(time > crop_start_millis) * (time < crop_end_millis)].values
    time = time[(time > crop_start_millis) * (time < crop_end_millis)].values
    
    # Resample
    if resample_fs is None:
        resampled = channel
        resampled_time = time
    else:
        ts = time[1] - time[0]
        secs = len(channel) * ts
        secs = secs/1000
        resampled = signal.resample(channel, int(secs*resample_fs))
        resampled_time = np.linspace(time[0], time[-1], len(resampled))
        fs = resample_fs

    # Remove line nosie.
    if filter_type == 'butter':
        resampled = butter_bandpass_filter(resampled, line_noise_band[0], line_noise_band[1], fs, 'bandstop')
        resampled_time = time
    else:
        df_filtered = blackman_harris_filter(resampled, resampled_time, line_noise_band, fs, numtaps=blackmann_harris_ntaps)
        resampled = df_filtered['channel']
        resampled_time = df_filtered['time']
    
    # PSD.
    if method == 'welch':
        # Welch method
        freq, power = signal.welch(resampled, fs, nperseg=4*fs)
        df_power = pd.DataFrame({'freq': freq, 'power': power})
    elif method == 'fft':
        # FFT method
        T = 1/fs
        N = len(resampled)
        yf = scipy.fftpack.fft(resampled)
        yf = 2 / (N/2) * np.abs(yf[:N//2])
        xf = np.linspace(0, 1/(2*T), N // 2)
        df_power = pd.DataFrame({'freq': xf, 'power': yf})
    elif method == 'pburg':
        # Burgs method
        order = min(len(resampled)-2, int(fs/4))
        p = pburg(resampled, order, sampling=fs, NFFT=fs)
        power = p.psd
        freq = np.linspace(0, fs/2, len(power))
        df_power = pd.DataFrame({'freq': freq, 'power': power})

    df_power = df_power[df_power['freq'] < fs/2]
    return df_power

# Diagnose

In [5]:
# ind = 10
# trials = read_trials_from_raw(paths_raw[10])
# trial = trials[10]
# time = trial['time']
# channel = trial['C3']

# fs = 256
# band = [8, 12]
# blackmann_harris_ntaps = 101
# line_noise_band = [58, 62]

# for filter_type in ['butter', 'blackmann']:
#     i = 0
#     f, axs = plt.subplots(1,5,figsize=(16,2))
#     axs[i].plot(time, channel)
#     i = i + 1
    
#     df_phase = get_phase(channel, time, band=band, fs=fs, filter_type=filter_type, start_time_ms=-750, stop_time_ms=-1, blackmann_harris_ntaps=blackmann_harris_ntaps)
#     axs[i].plot(df_phase['time'], df_phase['phase'])
#     axs[i].set_title("Phase - {}".format(filter_type))
#     i = i + 1
    
#     for method in ['pburg', 'welch', 'fft']:
#         df_power = get_power(channel, time, fs, crop_start_millis=-750, crop_end_millis=-1, method=method, filter_type=filter_type, resample_fs=None, line_noise_band=line_noise_band, blackmann_harris_ntaps=blackmann_harris_ntaps)
#         df_power = df_power[df_power['freq'] < 130]
#         axs[i].plot(df_power['freq'], df_power['power'])
#         axs[i].set_title("Power - {} - {}".format(filter_type, method))
#         i = i + 1
#     plt.tight_layout()
#     plt.show()

In [7]:
def process_trials(path_raw, path_raw_hjorth, path_clean, path_clean_hjorth):
    df_powers = []
    df_phases = []
    sub = path_raw.split('/')[-1].split('.')[0].split('_')[-1]
    trials_raw = read_trials_from_raw(path_raw)
    trials_raw_hjorth = read_trials_from_hjorth(path_raw_hjorth)
    trials_clean = read_trials_from_raw(path_clean)
    trials_clean_hjorth = read_trials_from_hjorth(path_clean_hjorth)
    blackmann_harris_ntaps = 101
    for trial_num in range(len(trials_raw)):
        trial_raw = trials_raw[trial_num]
        trial_raw_hjorth = trials_raw_hjorth[trial_num]
        trial_clean = trials_clean[trial_num]
        trial_clean_hjorth = trials_clean_hjorth[trial_num]
        trial_clean_avg = trial_clean.copy()
        trial_clean_avg['C3'] = trial_clean_avg[['FC5', 'FC3', 'FC1', 'C5', 'C3', 'C1', 'CP5', 'CP3', 'CP1']].mean(axis=1)
        trial_raw_avg = trial_raw.copy()
        trial_raw_avg['C3'] = trial_raw_avg[['FC5', 'FC3', 'FC1', 'C5', 'C3', 'C1', 'CP5', 'CP3', 'CP1']].mean(axis=1)
        for eeg_type, artifact_removed, trial in zip(['Raw', 'Hjorth', 'Raw', 'Hjorth', 'Average', 'Average'], [False, False, True, True, True, False], [trial_raw, trial_raw_hjorth, trial_clean, trial_clean_hjorth, trial_clean_avg, trial_raw_avg]):
            for filter_name, filter_code in zip(['Butterworth', 'Blackmann-Harris'], ['butter', 'blackmann']):
                
                # PSD
                for time in [-150, -750]:
                    for method_name, method_code in zip(['Welch', 'FFT', 'Burg'], ['welch', 'fft', 'pburg']):
                        df_power = get_power(trial['C3'], trial['time'], fs=256, crop_start_millis=time, crop_end_millis=-1, method=method_code, filter_type=filter_code, resample_fs=None, line_noise_band=[58, 62], blackmann_harris_ntaps=blackmann_harris_ntaps)
                        for band, fc1, fc2 in zip(['Theta', 'Mu', 'Beta', 'Gamma'], [3.5, 8, 13, 30], [8, 12, 30, 80]):
                            df_power2 = df_power[df_power['freq'] >= fc1]
                            df_power2 = df_power2[df_power2['freq'] < fc2]
                            power = df_power2['power'].mean(axis=0)
                            df_powers.append({
                                'sub': sub,
                                'trial': trial_num+1,
                                'EEG': eeg_type,
                                'ArtifactRemoved': artifact_removed,
                                'Filter': filter_name,
                                'Time': time,
                                'Method': method_name,
                                'Band': band,
                                'Power': power
                            })
                            
                # Phase
                for band, fc1, fc2 in zip(['Theta', 'Mu', 'Beta', 'Gamma'], [3.5, 8, 13, 30], [8, 12, 30, 80]):
                    phase = get_phase(trial['C3'], trial['time'], band=[fc1, fc2], fs=256, filter_type=filter_code, start_time_ms=-750, stop_time_ms=-1, blackmann_harris_ntaps=blackmann_harris_ntaps)
                    phase = phase.iloc[-1]['phase'] + 180
                    df_phases.append({
                        'sub': sub,
                        'trial': trial_num+1,
                        'EEG': eeg_type,
                        'ArtifactRemoved': artifact_removed,
                        'Filter': filter_name,
                        'Band': band,
                        'Phase': phase
                    })
    return (df_powers, df_phases)

In [ ]:
paths = [list(a) for a in zip(paths_raw, paths_raw_hjorth, paths_clean, paths_clean_hjorth)]
num_cores = multiprocessing.cpu_count() - 2
results = Parallel(n_jobs=num_cores)(delayed(process_trials)(path_raw, path_raw_hjorth, path_clean, path_clean_hjorth) for path_raw, path_raw_hjorth, path_clean, path_clean_hjorth in tqdm(paths, total=len(paths)))

In [ ]:
df_powers = pd.DataFrame()
df_phases = pd.DataFrame()
for df_power, df_phase in tqdm(results):
    df_powers = pd.concat([df_powers, pd.DataFrame(df_power)])
    df_phases = pd.concat([df_powers, pd.DataFrame(df_phase)])

In [ ]:
df_powers.to_csv('166-d3-powers.csv')

In [ ]:
df_phases.to_csv('166-d3-phases.csv')

In [ ]:
df_powers.head()

# Power - box plot

In [ ]:
df_powers = pd.read_csv('166-d3-powers.csv')
df_phases = pd.read_csv('166-d3-phases.csv')

In [ ]:
def_values_power = {
    'ArtifactRemoved': True,
    'EEG': 'Raw',
    'Filter': 'Butterworth',
    'Time': -150,
    'Method': 'Welch'
}
hue_order = {
    'ArtifactRemoved': [True, False],
    'EEG': ['Raw', 'Hjorth', 'Average'],
    'Filter': ['Butterworth', 'Blackmann-Harris'],
    'Time': [-150, -750],
    'Method': ['FFT', 'Welch', 'Burg']
}

def_values_phase = {
    'ArtifactRemoved': True,
    'EEG': 'Raw',
    'Filter': 'Butterworth'
}

In [ ]:
df_power2 = df_powers[df_powers['sub'] == 'S10']

# with plt.style.context(['science-raquib2']):
f, axs = plt.subplots(2,3,figsize=(12,8))
i = 0
j = 0
for variable in tqdm(list(def_values_power.keys())):    
    df_power3 = df_power2
    print('--------------')
    print(variable)
    print('--------------')
    for key, value in def_values_power.items():
        if key != variable:
            df_power3 = df_power3[df_power3[key] == value]
            print('Performing {} = {} for constant {}, shape = {}'.format(key, value, variable, df_power3.shape))

    sns.boxplot(x="Band", y="Power", hue=variable, data=df_power3, ax=axs[i, j], fliersize=0, hue_order=hue_order[variable])
    # axs[i, j].set_ylim([-25,30])
    plt.setp(axs[i, j].lines, color='k')
    # axs[i, j].get_legend().remove()
    if j == 0:
        axs[i, j].set_ylabel('Power (dB)')
    else:
        axs[i, j].set_ylabel('')
        axs[i, j].get_yaxis().set_ticks([])
    j = j + 1
    if j > 2:
        j = 0
        i = i + 1

plt.tight_layout()

# Phase - histogram

In [ ]:
df_phase2 = df_phases[df_phases['sub'] == 'S10']
colors = ['#EF5350', '#43A047', '#039BE5']
f, axs = plt.subplots(3, 4, figsize=(7,5))
i = 0
j = 0
color_ind = 0
for variable in tqdm(list(def_values_phase.keys())):
    df_phase3 = df_phase2.copy()
    for key, value in def_values_phase.items():
            if key != variable:
                df_phase3 = df_phase3[df_phase3[key] == value]
    for band in ['Theta', 'Mu', 'Beta', 'Gamma']:
        print('{}, Band: {}, Options: {}'.format(variable, band, df_phase3[variable].unique()))
        for option in df_phase3[variable].unique():
            df_phase4 = df_phase3[df_phase3['Band'] == band]
            df_phase4 = df_phase4[df_phase4[variable] == option]
            sns.distplot(df_phase4['Phase'], hist=False, color=colors[color_ind], kde_kws={"shade": True}, ax=axs[i, j])
            color_ind = color_ind + 1
            if i == 2:
                axs[i, j].set_xlabel('Phase (degrees)')
            else:
                axs[i, j].set_xlabel('')
            axs[i, j].set_xlim([0, 360])
            axs[i, j].set_ylim([0, 0.007])
            axs[i, j].set_xticks([0, 90, 180, 270, 360])
        if i == 0:
            axs[i, j].set_title(band)
        if j == 0:
            axs[i, j].set_ylabel('Density')
        else:
            axs[i, j].get_yaxis().set_visible(False)
        j = j + 1
        if j > 3:
            j = 0
        color_ind = 0
    i = i + 1

plt.tight_layout()

# Trial - power

In [ ]:
df_power3.columns.values

In [ ]:
df_power2 = df_powers[df_powers['sub'] == df_powers['sub'].unique()[5]]
df_power2 = df_power2[df_power2['Band'] == 'Beta']
with plt.style.context(['science-raquib2']):
    f, axs = plt.subplots(3, 2, figsize=(6.5,4))
    # f, axs = plt.subplots(2,3,figsize=(13, 8))
    i = 0
    j = 0
    for variable in tqdm(list(def_values_power.keys())):
        df_power3 = df_power2
        print('--------------')
        print(variable)
        print('--------------')
        for key, value in def_values_power.items():
            if key != variable:
                df_power3 = df_power3[df_power3[key] == value]
                print('Performing {} = {} for constant {}, shape = {}'.format(key, value, variable, df_power3.shape))

        df_power3 = df_power3[['trial', 'Power', variable]]
        sns.lineplot(x="trial", y="Power", hue=variable, data=df_power3, ax=axs[i, j], hue_order=hue_order[variable], palette=sns.color_palette("Set1", df_power3[variable].nunique()))
        # axs[i, j].set_ylim([-25,35])
        # plt.setp(axs[i, j].lines, color='k')
        # axs[i, j].get_legend().remove()
        axs[i,j].set_xlabel('Trial')
        if j == 0:
            axs[i, j].set_ylabel('Power (dB)')
        else:
            axs[i, j].set_ylabel('')
            axs[i, j].get_yaxis().set_ticks([])
        j = j + 1
        if j > 1:
            j = 0
            i = i + 1

    plt.tight_layout()